In [1]:
import os
import re
import spacy
import nltk
import unicodedata
import pandas as pd
from nltk.corpus import stopwords
from dotenv import load_dotenv
from langchain import hub
from langchain_community.document_loaders import CSVLoader
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_chroma import Chroma
from util import dir_management

In [2]:
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
llm = ChatGroq(model="llama3-8b-8192")

In [ ]:
nltk.download('stopwords')
nlp = spacy.load('pt_core_news_sm')
stop_words = set(stopwords.words('portuguese'))
csv_columns = ['product_name', 'site_category_lv2',
               'overall_rating', 'review_text']

In [4]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def remove_exclamations_and_periods(text):
    text = re.sub(r'[!.,@]', '', text)
    return text


def remove_stop_words(text):
    return ' '.join([word for word in text.split() if word not in stop_words])


def remove_accents(text):
    text = unicodedata.normalize('NFD', text)
    text = re.sub(r'[\u0300-\u036f]', '', text)
    return text


def format_docs(docs):
    return '\n\n'.join(doc.page_content for doc in docs)


In [ ]:
rows_number = 5000  # Define quantas rows do csv serão utilizadas no RAG
df = pd.read_csv('../B2W-Reviews.csv')
df_reduced = df.drop(
    columns=[col for col in df.columns if col not in csv_columns])
for column in csv_columns:
    df_reduced[column] = df_reduced[column].apply(
        lambda x: clean_text(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_exclamations_and_periods(str(x)))
    df_reduced[column] = df_reduced[column].apply(
        lambda x: remove_accents(str(x)))
    
    
result_file_name = f'B2W-Reviews-top{rows_number}.csv'
new_df = df_reduced.head(rows_number).to_csv(os.path.join
                                             (dir_management.get_out_dir(),
                                              result_file_name))

In [25]:
loader = CSVLoader(file_path=os.path.join(dir_management.get_out_dir(),
                                          result_file_name),
                   encoding='utf-8',
                   csv_args={
                       'delimiter': ',',
                       'quotechar': '"',
                       'fieldnames': csv_columns
})

docs = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
model_name = 'sentence-transformers/all-MiniLM-L6-v2'
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
vectorstore = Chroma.from_documents(
    documents=splits, embedding=hf)

retriever = vectorstore.as_retriever()

In [8]:
prompt_template = """

Responda sempre de forma clara e precisa em português do Brasil.
Você é um assistente especializado em marketing e feedback de clientes. Responda perguntas que estejam relacionadas a marketing, campanhas, review de clientes, avaliações de produtos ou publicidade. 
Para perguntas fora desse escopo, responda: "Essa pergunta está fora do escopo deste chatbot. Por favor, faça perguntas relacionadas a marketing."
Para perguntas sobre produtos, use apenas o feedback dos clientes fornecido no contexto para responder, não invente respostas. Se o contexto não tiver informações suficientes, responda: "Não há informações suficientes para responder a essa pergunta."

Contexto: {context}
Pergunta: {question}

Resposta:

"""

In [9]:
def custom_prompt(context, question):
    return prompt_template.format(context=context, question=question)

# RAG Chain com prompt customizado 
def run_rag_chain(question):
    # Recupera documentos e formata o contexto
    retrieved_docs = retriever.invoke(question)# aumentar para k=10 para ver o resultado 
    formatted_context = format_docs(retrieved_docs)
    
    # Cria o prompt customizado
    full_prompt = custom_prompt(formatted_context, question)
    
    # Passa o prompt para o modelo de linguagem
    response = llm.invoke(full_prompt)
    
    # Parseia a resposta para o formato correto
    parsed_response = StrOutputParser().parse(response)
    
    return parsed_response.content

In [ ]:
response = run_rag_chain('Me fale sobre o clima de amanhã.')
print(response)

In [ ]:
response = run_rag_chain('Me fale sobre o Samsung Galaxy S24')
print(response)

In [ ]:
response = run_rag_chain ("Pode me indicar bons smartphones?")
print(response)                    

perguntas para marketing:

Quais são os principais pontos de satisfação mencionados pelos clientes em suas avaliações?
Quais são os principais motivos de insatisfação dos clientes sobre os produtos?
Existe alguma tendência nos feedbacks que indica um aumento ou diminuição da satisfação do cliente?

Quais características dos produtos são mais frequentemente elogiadas pelos clientes?
Como os produto se comparam a concorrentes nas avaliações dos clientes?
Quais produtos têm recebido as melhores e as piores avaliações, e por quê?

Com base nos feedbacks, que ações de marketing podemos implementar para melhorar a percepção da marca?
Quais campanhas anteriores parecem ter gerado mais impacto positivo, de acordo com as avaliações?
Como podemos usar os feedbacks dos clientes para criar uma nova campanha de marketing?

Existem padrões nas avaliações que indicam diferentes preferências entre diferentes grupos de clientes (por exemplo, idade, gênero, localização)?
Que tipo de clientes estão mais satisfeitos com nosso produto e por quê?

Quais tendências recentes podem ser observadas nos feedbacks que poderiam impactar nossas estratégias de marketing futuras?
O que os clientes estão buscando atualmente que pode não estar sendo atendido pelos nossos produtos?

Como podemos melhorar a experiência do cliente com base nas avaliações recebidas?
Quais atributos dos produtos mais impactam a decisão de compra, de acordo com as opiniões dos clientes?
Os clientes recomendariam o nosso produto a um amigo? Qual é a porcentagem de respostas positivas?

Qual o produto mais bem avaliado pelos clientes ?
Recomendações para melhorar a satisfação do cliente?
Qual é o feedback positivos dos clientes sobre os produtos?

s campanhas anteriores parecem ter gerado mais impacto positivo, de acordo com as avaliações?
Como podemos usar os feedbacks dos clientes para criar uma nova campanha de marketing?

Existem padrões nas avaliações que indicam diferentes preferências entre diferentes grupos de clientes (por exemplo, idade, gênero, localização)?
Que tipo de clientes estão mais satisfeitos com nosso produto e por quê?

Quais tendências recentes podem ser observadas nos feedbacks que poderiam impactar nossas estratégias de marketing futuras?
O que os clientes estão buscando atualmente que pode não estar sendo atendido pelos nossos produtos?

Como podemos melhorar a experiência do cliente com base nas avaliações recebidas?
Quais atributos dos produtos mais impactam a decisão de compra, de acordo com as opiniões dos clientes?
Os clientes recomendariam o nosso produto a um amigo? Qual é a porcentagem de respostas positivas?

Qual o produto mais bem avaliado pelos clientes ?
Recomendações para melhorar a satisfação do cliente?
Qual é o feedback positivos dos clientes sobre os produtos?